# 使用AutoLM实现Titanic项目

项目使用aws的autogluon框架自动微调项目，由于Tabular类型数据不支持gpu操作。

In [ ]:
!pip install -U autogluon > /dev/null

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

train_data = TabularDataset('/kaggle/input/titanic/train.csv')
label = 'Survived'

# train the model
predictor = TabularPredictor(label=label).fit(train_data, presets="best_quality", num_gpus=2)

In [ ]:
# 合并label

import pandas as pd 

df_test = pd.read_csv("/kaggle/input/titanic/test.csv")
df_lab = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

test_data = pd.merge(df_test, df_lab, on="PassengerId")

In [ ]:
# label_data = TabularDataset('/kaggle/working/label.csv')

label = 'Survived'
y_pred = predictor.predict(test_data.drop(columns=[label]))
y_pred.head()

In [ ]:
predictor.evaluate(test_data, silent=True)

In [ ]:
# 各种ml模型的分数排名
predictor.leaderboard(test_data)


In [ ]:
# 导出结果
df_submission = test_data[['PassengerId', 'Survived']]
df_submission.assign(Survived=y_pred)

df_submission.to_csv("submission.csv", index=False)